In [1]:
import pandas as pd 

from time import perf_counter
from langchain_community.llms import GPT4All
from langchain_core.prompts import PromptTemplate

In [2]:
MODEL_PATH = r'D:\Programs\GPT4ALL DOWNLOAD\mistral-7b-instruct-v0.2.Q4_K_M.gguf'
# MODEL_PATH = r'D:\Programs\GPT4ALL DOWNLOAD\qwen2-1_5b-instruct-q4_0.gguf'

In [3]:
%%time 

model = GPT4All(model=MODEL_PATH, 
                device='kompute', 
                verbose=True) # cuda - kompute - nvidia - gpu

CPU times: total: 969 ms
Wall time: 7.91 s


In [14]:
data_from_user = {"destination": "Saudi Arabia",
                  "start_date": "August-15-2024",
                  "end_date": "August-20-2024",
                  "activities": ["Shopping", "Photography", "Unique Experiences", "Museums"],
                  "person_status": "Single",
                  "budget": "Low"}

In [15]:
places = pd.read_excel('places.xlsx')
places.head(1)

,Places,Activity
0,Al-Ula,Sightseeing


In [16]:
chosen_activity = data_from_user["activities"]
chosen_places = places[places['Activity'].isin(chosen_activity)]
chosen_places.head()

,Places,Activity
60,"Kingdom Centre Mall, Riyadh",Shopping
61,"Al Nakheel Mall, Riyadh",Shopping
62,"Red Sea Mall, Jeddah",Shopping
63,"Mall of Arabia, Jeddah",Shopping
64,"Al Khobar Mall, Al Khobar",Shopping


In [17]:
list_of_tuples = list(chosen_places.itertuples(index=False, name=None))

In [18]:
trip_data = {"destination": "Saudi Arabia",
             "start_date": "August-15-2024",
             "end_date": "August-20-2024",
             "activities": ["Shopping", "Photography", "Unique Experiences", "Museums"],
             "person_status": "Single",
             "budget": "Low",
             "places": list_of_tuples}

In [19]:
TRIP_PLANNER_PROMPT_TEMPLATE = """[INST]
You are a smart trip planner. You plan trips based user data like dates, activities, places, single or couple or famliy.
Spread the activities logically over the days and include travel time between locations if necessary. 
Include more than one activity each day when appropriate.
The activities should match those listed above. 

Plan a trip itinerary with the following details:

Destination: {destination}
Dates: {start_date} to {end_date}
Person Status: {person_status}
Budget: {budget}

Places and activities: {places}

Return only the itinerary in CSV format with columns for Day, Date, Activity, and Location. 
Do not return any other text.

Example format:
1,August-15-2023,Arrival,King Abdulaziz International Airport,Riyadh,Saudi Arabia
2,August-16-2023,Shopping,Mall of Riyadh,Riyadh
[/INST]
"""

In [20]:
TRIP_PLANNER_PROMPT_TEMPLATE = """
You are a smart trip planner. You plan trips based user data like dates, activities, places, single or couple or famliy.
Spread the activities logically over the days and include travel time between locations if necessary. 
Include more than one activity each day when appropriate.
The activities should match those listed above. 

Plan a trip itinerary with the following details:

Destination: {destination}
Dates: {start_date} to {end_date}
Person Status: {person_status}
Budget: {budget}

Places and activities: {places}

Return only the itinerary in CSV format with columns for Day, Date, Activity, and Location. 
Do not return any general text in other format.

Example format:
1,August-15-2023,Arrival,King Abdulaziz International Airport,Riyadh,Saudi Arabia
2,August-16-2023,Shopping,Mall of Riyadh,Riyadh
"""

In [21]:
trip_prompt = PromptTemplate.from_template(template = TRIP_PLANNER_PROMPT_TEMPLATE)

print( trip_prompt.format(**trip_data))


You are a smart trip planner. You plan trips based user data like dates, activities, places, single or couple or famliy.
Spread the activities logically over the days and include travel time between locations if necessary. 
Include more than one activity each day when appropriate.
The activities should match those listed above. 

Plan a trip itinerary with the following details:

Destination: Saudi Arabia
Dates: August-15-2024 to August-20-2024
Person Status: Single
Budget: Low

Places and activities: [('Kingdom Centre Mall, Riyadh', 'Shopping'), ('Al Nakheel Mall, Riyadh', 'Shopping'), ('Red Sea Mall, Jeddah', 'Shopping'), ('Mall of Arabia, Jeddah', 'Shopping'), ('Al Khobar Mall, Al Khobar', 'Shopping'), ('Dhahran Mall, Dhahran', 'Shopping'), ('Riyadh Gallery, Riyadh', 'Shopping'), ('Granada Mall, Riyadh', 'Shopping'), ('Jeddah Mall, Jeddah', 'Shopping'), ('The Boulevard, Riyadh', 'Shopping'), ('Aziz Mall, Jeddah', 'Shopping'), ('Al Haramain Mall, Makkah', 'Shopping'), ('Makarim Mall

In [22]:
def set_model_params(llm,
                    max_tokens: int = 256,
                    temp: float = 0.7,
                    top_k: int = 40,
                    top_p: float = 0.1,
                    streaming = False): 
    llm.max_tokens = max_tokens
    llm.temp = temp
    llm.top_k = top_k
    llm.top_p = top_p
    llm.streaming = streaming
    
    return llm 

In [23]:
model = set_model_params(model, max_tokens=1024, temp=0.1)

In [24]:
chain = trip_prompt | model

In [25]:
%%time

response = chain.invoke(trip_data)

CPU times: total: 5.05 s
Wall time: 1min 4s


In [26]:
print( response )

3,August-17-2023,Photography,Hegra (Mada'in Salih),Riyadh

Note: The dates in the example are fictional and should be replaced with actual dates.
Here is a sample itinerary for your trip to Saudi Arabia from August 15th to August 20th, 2024:
```
Date, Activity, Location
August-15-2023, Arrival, King Abdulaziz International Airport, Riyadh, Saudi Arabia
August-16-2023, Shopping, Kingdom Centre Mall, Riyadh
August-17-2023, Photography, Edge of the World, Riyadh
August-18-2023, Unique Experiences, Al-Masjid al-Haram, Makkah
August-19-2023, Photography, Farasan Islands, Jeddah
August-20-2023, Shopping, Dhahran Corniche, Dhahran
```

Note: The dates in the example are fictional and should be replaced with actual dates. You can replace them with your own travel plans.
I have created a sample itinerary for you based on the details provided:
``


In [29]:
def csv_parser(response, max_split=3, n_columns=4):
    # try split by new line.
    table = []
    try:
        lines = response.split('\n')
        # check each line is empty or not.
        for line in lines:
            if len(line) == 0:
                continue
            try:
                row = line.split(',', maxsplit=max_split)
                # add validator 
                if len(row) == n_columns:
                    table.append(row)
            except Exception as e:
                print(f'[ERROR] can not split each line by , --- {e}')
    except Exception as e:
        print(f'[ERROR] can not split response by \n --- {e}')

    return table
    
# csv_parser(test)

In [30]:
dummy_table = csv_parser(response, max_split=2, n_columns=3)
dummy_table

[['3', 'August-17-2023', "Photography,Hegra (Mada'in Salih),Riyadh"],
 ['Date', ' Activity', ' Location'],
 ['August-15-2023',
  ' Arrival',
  ' King Abdulaziz International Airport, Riyadh, Saudi Arabia'],
 ['August-16-2023', ' Shopping', ' Kingdom Centre Mall, Riyadh'],
 ['August-17-2023', ' Photography', ' Edge of the World, Riyadh'],
 ['August-18-2023', ' Unique Experiences', ' Al-Masjid al-Haram, Makkah'],
 ['August-19-2023', ' Photography', ' Farasan Islands, Jeddah'],
 ['August-20-2023', ' Shopping', ' Dhahran Corniche, Dhahran']]

In [32]:
trip_table = pd.DataFrame(dummy_table, columns=["Day", "Date", "Activity", "Location"])

In [33]:
trip_table

,Day,Date,Activity,Location
0,3,August-17-2023,Photography,"Hegra (Mada'in Salih),Riyadh"
1,August-15-2023,Arrival,King Abdulaziz International Airport,"Riyadh, Saudi Arabia"
2,August-16-2023,Shopping,Kingdom Centre Mall,Riyadh
3,August-17-2023,Photography,Edge of the World,Riyadh
4,August-18-2023,Unique Experiences,Al-Masjid al-Haram,Makkah
5,August-19-2023,Photography,Farasan Islands,Jeddah
6,August-20-2023,Shopping,Dhahran Corniche,Dhahran


In [35]:
# trip_table.to_csv('results.csv', index=False)

In [ ]:
def get_trip_plan(trip_data):
    
    
    return 0